This notebook is a simple sanity check for the wrapper of the se3dif algorithm using simulated data.

In [ ]:
from pathlib import Path

import yaml
import trimesh
import numpy as np

from se3dif.visualization import create_gripper_marker

from grasping_benchmarks.algorithms.se3dif import Se3DifGraspPlanner
from grasping_benchmarks.base import CameraData
from grasping_benchmarks.base.transformations import rotation_and_position_to_homogeneous_matrix

In [ ]:
ACRONYM_DATASET_PATH = Path("/home/data/")

In [ ]:
pointcloud = np.load(
    list(
        (
            Path.cwd().parent
            / "test_data"
            / "pointcloud_segmented_points"
            / "011_banana"
        ).glob("*.npy")
    )[0]
)

pointcloud.shape

In [ ]:
N_POINTS = 1000
TARGET_MAX_SIZE = 0.3*8 # 0.3 is roughly the average max extent of the objects in the dataset

pointcloud = pointcloud[np.random.choice(len(pointcloud), N_POINTS)]
pointcloud_offset = np.mean(pointcloud, axis=0)
pointcloud -= pointcloud_offset

pointcloud_extents = np.max(pointcloud, axis=0) - np.min(pointcloud, axis=0)
scale_factor = TARGET_MAX_SIZE / np.max(pointcloud_extents)
pointcloud *= scale_factor

In [ ]:
with open(Path.cwd().parent / "cfg" / "base_config.yaml", "r") as f:
    cfg = yaml.load(f)

planner = Se3DifGraspPlanner(cfg)

In [ ]:
camera_data = CameraData(pointcloud=pointcloud)

In [ ]:
grasps = planner.plan_grasps(camera_data)

In [ ]:
def visualize_grasp_axis(H_grasp, gripper_height=None):
    gripper_height = gripper_height or 0.02

    grasp_point = trimesh.primitives.Sphere(radius=0.004, center=H_grasp[:3, 3])

    # assumption: grasp axis is along z-axis of gripper coordinate system
    grasp_vector = trimesh.creation.cylinder(
        radius=0.002,
        segment=(H_grasp[:3, 3], H_grasp[:3, 3] + gripper_height * H_grasp[:3, 2]),
    )

    return trimesh.util.concatenate([grasp_point, grasp_vector])


scene = trimesh.Scene()
for point in pointcloud:
    scene.add_geometry(trimesh.primitives.Sphere(radius=0.02, center=point))

for grasp in grasps:
    H_grasp = rotation_and_position_to_homogeneous_matrix(
        grasp.rotation, grasp.position
    )
    scene.add_geometry(create_gripper_marker().apply_transform(H_grasp))
    scene.add_geometry(visualize_grasp_axis(H_grasp, 1))

scene.show()